```
Reads CSV file
- returns list of dicts, one per row
Reads list of dicts, one per row
- If row has no propertyID, row is ignored (for now, drop "shape on own line")
- Otherwise (ie, row has propertyID):
  - If FIRST_ROW_ENCOUNTERED is True:
    - start = True
    - If shapeID is not specified 
      - Assign default shapeID
      - csvshape_dict["shapeID"] = shapeID (either exists or was assigned default)
      - csvshape_dict["shapeLabel"] = shapeLabel (use if exists)
      - csvshape_dict["shapeClosed"] = shapeClosed 
      - csvshape_dict["shapeFirst"] = shapeFirst
      - csvshape_dict["start"] = True
    - FIRST_ROW_ENCOUNTERED = False

    - If shapeID in 
    - Append csvshape_dict to csvshape_dicts_list
    - Clear csvshape_dict
    - Instantiate CSVShape as cs
 

  - If FIRST_ROW_ENCOUNTERED is False:
    - If shapeID is NOT specified 
      - shapeID = csvshape_dicts_list[-1] # last item
      - And shapeID exists in shapeid_dict
        - Use it
      - And shapeID does not yet exist in shapeid_dict
        - Append to shapeid_dict:
          - { "shapeID": (shapeLabel, shapeClosed, shapeFirst) }
    - Instantiates CSVShape
      - If CSVShape has a valid value for shapeID, this is used
      - If CSVShape does not have a shapeID
        - Gets latest shapeID from shapeids_list
        - For rest of the fields in CSVShape:
          - Assigns values (if present) to CSVShape
      - CSVShape validates itself
```

### Import

In [39]:
from dataclasses import dataclass, field, asdict
from typing import List
from csv2shex.csvreader import (
    csvreader, 
    _get_csvrow_dicts_list,
    _get_corrected_csvrows_list, 
    _get_csvshape_dicts_list, 
)
from csv2shex.csvrow import CSVRow
from csv2shex.utils import pprint_df
import pandas as pd

### Declare

In [40]:
@dataclass
class CSVTripleConstraint:
    """Instances hold TAP/CSV row elements that form a triple constraint."""
    propertyID: str = None
    valueConstraint: str = None
    valueShape: str = None
    extras: field(default_factory=dict) = None
    # propertyLabel: str = None
    # mandatory: str = None
    # repeatable: str = None
    # valueNodeType: str = None
    # valueDataType: str = None
    # valueConstraintType: str = None
    # note: str = None

@dataclass
class CSVShape:
    """Instances hold TAP/CSV row elements that form a shape."""
    shapeID: str = None
    shapeLabel: str = None
    shapeClosed: str = None
    start: bool = False
    tripleconstraints_list: List[CSVTripleConstraint] = field(default_factory=list)

@dataclass
class CSVSchema:
    """Set of shapes."""

### For each row
#### 1. On finding new shapeID, capture shape-related elements in a shape_dict.

In [41]:
shape_dict = dict()
shape_dict["shapeID"] = "b"
shape_dict["shapeLabel"] = "label"
shape_dict["shapeClosed"] = False
shape_dict["start"] = True
shape_dict["tripleconstraints_list"] = list()
shape_dict

{'shapeID': 'b',
 'shapeLabel': 'label',
 'shapeClosed': False,
 'start': True,
 'tripleconstraints_list': []}

#### 2. Initialize CSVShape instance with shape_dict, attach to dict_of_shape_dicts.

In [42]:
dict_of_shape_objs = dict()
dict_of_shape_objs[shape_dict["shapeID"]] = CSVShape(**shape_dict)
dict_of_shape_objs

{'b': CSVShape(shapeID='b', shapeLabel='label', shapeClosed=False, start=True, tripleconstraints_list=[])}

In [43]:
dict_of_shape_objs["b"]

CSVShape(shapeID='b', shapeLabel='label', shapeClosed=False, start=True, tripleconstraints_list=[])

In [44]:
"b" in dict_of_shape_objs

True

In [45]:
# List of triple constraint dicts for shapeID is addressable.
dict_of_shape_objs["b"].tripleconstraints_list

[]

#### 3. On finding new shape, add to dict_of_shape_dicts.

In [46]:
shape_dict = dict()
shape_dict["shapeID"] = "c"
shape_dict["shapeLabel"] = "clabel"
shape_dict["shapeClosed"] = False
shape_dict["start"] = False
shape_dict["tripleconstraints_list"] = list()

In [47]:
dict_of_shape_objs

{'b': CSVShape(shapeID='b', shapeLabel='label', shapeClosed=False, start=True, tripleconstraints_list=[])}

In [48]:
dict_of_shape_objs[shape_dict["shapeID"]] = CSVShape(**shape_dict)
dict_of_shape_objs

{'b': CSVShape(shapeID='b', shapeLabel='label', shapeClosed=False, start=True, tripleconstraints_list=[]),
 'c': CSVShape(shapeID='c', shapeLabel='clabel', shapeClosed=False, start=False, tripleconstraints_list=[])}

In [49]:
dict_of_shape_objs.keys()

dict_keys(['b', 'c'])

In [50]:
# After first row, for rows that lack shapeIDs, get most-recently-inserted key from dict_of_shape_dicts
list(dict_of_shape_objs.keys())[-1]

'c'

#### 4.

In [51]:
# Problem: append multiple triple constraint dicts to tripleconstraints_list
tc_dict = dict()
tc_dict["propertyID"] = "dc:type"
tc_dict["valueConstraint"] = "foaf:Person"
dict_of_shape_objs["b"].tripleconstraints_list.append(tc_dict)
dict_of_shape_objs

{'b': CSVShape(shapeID='b', shapeLabel='label', shapeClosed=False, start=True, tripleconstraints_list=[{'propertyID': 'dc:type', 'valueConstraint': 'foaf:Person'}]),
 'c': CSVShape(shapeID='c', shapeLabel='clabel', shapeClosed=False, start=False, tripleconstraints_list=[])}

In [52]:
# Problem: append multiple triple constraint dicts to tripleconstraints_list
tc_dict = dict()
tc_dict["propertyID"] = "dc:creator"
tc_dict["valueConstraint"] = "http://example.org/person1"
tc_obj = CSVTripleConstraint(**tc_dict)
tc_obj

CSVTripleConstraint(propertyID='dc:creator', valueConstraint='http://example.org/person1', valueShape=None, extras=None)

In [53]:
CSVTripleConstraint(**tc_dict)
dict_of_shape_objs

{'b': CSVShape(shapeID='b', shapeLabel='label', shapeClosed=False, start=True, tripleconstraints_list=[{'propertyID': 'dc:type', 'valueConstraint': 'foaf:Person'}]),
 'c': CSVShape(shapeID='c', shapeLabel='clabel', shapeClosed=False, start=False, tripleconstraints_list=[])}

In [54]:
# This is to pretty-print the entire CSVShape
vars(CSVShape(shapeID='b', shapeLabel='label', shapeClosed=False, start=True, tripleconstraints_list=[
{'propertyID': 'dc:type', 'valueConstraint': 'foaf:Person'}, 
{'propertyID': 'dc:creator', 'valueConstraint': 'http://example.org/person1'}]))

{'shapeID': 'b',
 'shapeLabel': 'label',
 'shapeClosed': False,
 'start': True,
 'tripleconstraints_list': [{'propertyID': 'dc:type',
   'valueConstraint': 'foaf:Person'},
  {'propertyID': 'dc:creator',
   'valueConstraint': 'http://example.org/person1'}]}